In [1]:
import torch
import torch.nn as nn
from torch import optim
import time, random
import os
from tqdm import tqdm
from lstm import LSTMSentiment
from bilstm import BiLSTMSentiment
from torchtext import data
import numpy as np
import argparse
from sklearn.metrics import accuracy_score,f1_score

torch.set_num_threads(8)
torch.manual_seed(1)
random.seed(1)





def get_accuracy(truth, pred):
    assert len(truth) == len(pred)
    return accuracy_score(truth,pred)

def get_f1(truth,pred):
    assert len(truth) == len(pred)
    return f1_score(truth,pred,average='weighted')

def train_epoch_progress(model, train_iter, loss_function, optimizer, text_field, label_field, epoch):
    model.train()
    avg_loss = 0.0
    truth_res = []
    pred_res = []
    count = 0
    for batch in tqdm(train_iter, desc='Train epoch '+str(epoch+1)):
        sent, label = batch.text, batch.label
        label.data.sub_(1)
        truth_res += list(label.data)
        model.batch_size = len(label.data)
        model.hidden = model.init_hidden()
        pred = model(sent)
        pred_label = pred.data.max(1)[1].numpy()
        pred_res += [x for x in pred_label]
        model.zero_grad()
        loss = loss_function(pred, label)
        avg_loss += loss.data[0]
        count += 1
        loss.backward()
        optimizer.step()
    avg_loss /= len(train_iter)
    acc = get_accuracy(truth_res, pred_res)
    f1 = get_f1(truth_res, pred_res)
    return avg_loss, acc,f1


def train_epoch(model, train_iter, loss_function, optimizer):
    model.train()
    avg_loss = 0.0
    truth_res = []
    pred_res = []
    count = 0
    for batch in train_iter:
        sent, label = batch.text, batch.label
        label.data.sub_(1)
        truth_res += list(label.data)
        model.batch_size = len(label.data)
        model.hidden = model.init_hidden()
        pred = model(sent)
        pred_label = pred.data.max(1)[1].numpy()
        pred_res += [x for x in pred_label]
        model.zero_grad()
        loss = loss_function(pred, label)
        avg_loss += loss.data[0]
        count += 1
        loss.backward()
        optimizer.step()
    avg_loss /= len(train_iter)
    acc = get_accuracy(truth_res, pred_res)
    f1 = get_f1(truth_res, pred_res)
    return avg_loss, acc,f1


def evaluate(model, data, loss_function, name):
    model.eval()
    avg_loss = 0.0
    truth_res = []
    pred_res = []
    for batch in data:
        sent, label = batch.text, batch.label
        label.data.sub_(1)
        truth_res += list(label.data)
        model.batch_size = len(label.data)
        model.hidden = model.init_hidden()
        pred = model(sent)
        pred_label = pred.data.max(1)[1].numpy()
        pred_res += [x for x in pred_label]
        loss = loss_function(pred, label)
        avg_loss += loss.data[0]
    avg_loss /= len(data)
    acc = get_accuracy(truth_res, pred_res)
    f1 = get_f1(truth_res, pred_res)
    print(name + ': loss %.2f acc %.1f f1 %.2f' % (avg_loss, acc*100,f1))
    return acc,f1


def load_sst(text_field, label_field, batch_size):
    train, dev, test = data.TabularDataset.splits(path='./data/SST2/', train='train.csv',
                                                  validation='dev.csv', test='test.csv', format='csv',
                                                  fields=[('text', text_field), ('label', label_field)])
    text_field.build_vocab(train, dev, test)
    label_field.build_vocab(train, dev, test)
    train_iter, dev_iter, test_iter = data.BucketIterator.splits((train, dev, test),
                batch_sizes=(batch_size, len(dev), len(test)), sort_key=lambda x: len(x.text), repeat=False, device=-1)
    return train_iter, dev_iter, test_iter


# def adjust_learning_rate(learning_rate, optimizer, epoch):
#     lr = learning_rate * (0.1 ** (epoch // 10))
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr
#     return optimizer





In [2]:
EPOCHS = 10
USE_GPU = torch.cuda.is_available()
EMBEDDING_DIM = 300
HIDDEN_DIM = 150

BATCH_SIZE = 5
timestamp = str(int(time.time()))
best_dev_acc = 0.0


text_field = data.Field(lower=True)
label_field = data.Field(sequential=False)
train_iter, dev_iter, test_iter = load_sst(text_field, label_field, BATCH_SIZE)



In [3]:
vars(label_field.vocab)

{'freqs': Counter({'0': 19347,
          '1': 9808,
          '2': 730,
          '3': 153,
          '4': 906,
          '5': 1187,
          'label': 3}),
 'itos': ['<unk>', '0', '1', '5', '4', '2', '3', 'label'],
 'stoi': defaultdict(<function torchtext.vocab._default_unk_index>,
             {'0': 1,
              '1': 2,
              '2': 5,
              '3': 6,
              '4': 4,
              '5': 3,
              '<unk>': 0,
              'label': 7}),
 'vectors': None}

In [4]:
#load dictionary
import pickle
emoji_dict =pickle.load(open('./data/SST2/emoji_dict.p','rb'))
print("emoji dictionary load successfully")
# load embedding
word_to_idx = text_field.vocab.stoi
pretrained_embeddings = np.random.uniform(-0.25, 0.25, (len(text_field.vocab), 300))
pretrained_embeddings[0] = 0
#word2vec = load_bin_vec('./data/GoogleNews-vectors-negative300.bin', word_to_idx)
from gensim.models.keyedvectors import KeyedVectors
word2vec= KeyedVectors.load_word2vec_format('./embedding/GoogleNews-vectors-negative300.bin', binary=True)
for word in word2vec.vocab:
    pretrained_embeddings[word_to_idx[word]-1] = word2vec[word]
print("word2vec load successfully")
emoji2vec =KeyedVectors.load_word2vec_format('./embedding/emoji2vec.bin', binary=True)
for word in emoji2vec.vocab:
    if word in emoji_dict.keys():
        pretrained_embeddings[word_to_idx[emoji_dict[word]]-1] = emoji2vec[word]
print("emoji2vec load successfully")
emoticon2vec = KeyedVectors.load_word2vec_format('./embedding/emoticon2vec.txt', binary=False)



for word in emoticon2vec.vocab:
    pretrained_embeddings[word_to_idx[word]-1] = emoticon2vec[word]
print("emoticon2vec load successfully")    
print('Loading complete')

emoji dictionary load successfully
word2vec load successfully
emoji2vec load successfully
emoticon2vec load successfully
Loading complete


In [5]:
model = BiLSTMSentiment(embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, vocab_size=len(text_field.vocab), label_size=len(label_field.vocab)-1,\
                          use_gpu=USE_GPU, batch_size=BATCH_SIZE)


In [6]:
model.embeddings.weight.data.copy_(torch.from_numpy(pretrained_embeddings))




 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.1125  0.1872  0.0752  ...  -0.1721  0.1176  0.2066
 0.0801  0.1050  0.0498  ...   0.0037  0.0476 -0.0688
          ...             ⋱             ...          
 0.2044  0.1121  0.2245  ...  -0.2238 -0.1412 -0.0477
 0.1662  0.0474  0.2463  ...   0.1013 -0.1395 -0.0925
-0.0400  0.0009 -0.0988  ...   0.0423 -0.0371 -0.0146
[torch.FloatTensor of size 51322x300]

In [7]:
#define optimizer and loss function
best_model = model
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_function = nn.NLLLoss()



In [8]:
print('Training...')
out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
print("Writing to {}\n".format(out_dir))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
for epoch in range(EPOCHS):
    avg_loss, acc,f1 = train_epoch_progress(model, train_iter, loss_function, optimizer, text_field, label_field, epoch)
    tqdm.write('Train: loss %.2f acc %.1f f1 %.3f' % (avg_loss, acc*100,f1))
    dev_acc , dev_f1= evaluate(model, dev_iter, loss_function, 'Dev')
    if dev_acc > best_dev_acc:
        if best_dev_acc > 0:
            os.system('rm '+ out_dir + '/best_model' + '.pth')
        best_dev_acc = dev_acc
        best_model = model
        torch.save(best_model.state_dict(), out_dir + '/best_model' + '.pth')
        # evaluate on test with the best dev performance model
        test_acc,test_f1 = evaluate(best_model, test_iter, loss_function, 'Test')
test_acc,test_f1 = evaluate(best_model, test_iter, loss_function, 'Final Test')




Train epoch 1:   0%|          | 0/4049 [00:00<?, ?it/s]

Training...
Writing to /Users/nihaozheng/Desktop/NLP/project/model/pytorch-sentiment-classification-master/runs/1523740546



/Users/nihaozheng/Desktop/NLP/project/model/pytorch-sentiment-classification-master/bilstm.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_probs = F.log_softmax(y)
Train epoch 1: 100%|██████████| 4049/4049 [1:11:53<00:00,  1.07s/it]


Train: loss 0.84 acc 67.1 f1 0.638


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:89: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

Train: loss 0.08 acc 95.3 f1 0.953


Train epoch 10:   0%|          | 0/4049 [00:00<?, ?it/s]

Dev: loss 1.02 acc 74.0 f1 0.74


Train epoch 10: 100%|██████████| 4049/4049 [1:02:16<00:00,  1.08it/s]


Train: loss 0.08 acc 95.6 f1 0.956
Dev: loss 1.06 acc 72.3 f1 0.73
Final Test: loss 1.12 acc 68.8 f1 0.70


# try different dim of hidden layer (after load dictionary)

In [9]:
EPOCHS = 10
USE_GPU = torch.cuda.is_available()
EMBEDDING_DIM = 300
HIDDEN_DIM = 500

BATCH_SIZE = 5
timestamp = str(int(time.time()))
best_dev_acc = 0.0


text_field = data.Field(lower=True)
label_field = data.Field(sequential=False)
train_iter, dev_iter, test_iter = load_sst(text_field, label_field, BATCH_SIZE)




In [10]:
model = LSTMSentiment(embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, vocab_size=len(text_field.vocab), label_size=len(label_field.vocab)-1,\
                          use_gpu=USE_GPU, batch_size=BATCH_SIZE)



In [11]:
model.embeddings.weight.data.copy_(torch.from_numpy(pretrained_embeddings))





 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
-0.1564  0.1313 -0.0573  ...   0.2062 -0.0458  0.1574
 0.0801  0.1050  0.0498  ...   0.0037  0.0476 -0.0688
          ...             ⋱             ...          
 0.1075  0.0449  0.2172  ...  -0.1078  0.1145  0.2146
-0.2470 -0.1418 -0.0502  ...  -0.0088 -0.0527 -0.1548
-0.0400  0.0009 -0.0988  ...   0.0423 -0.0371 -0.0146
[torch.FloatTensor of size 51322x300]

In [12]:
#define optimizer and loss function
best_model = model
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_function = nn.NLLLoss()




In [ ]:
print('Training...')
out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
print("Writing to {}\n".format(out_dir))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
for epoch in range(EPOCHS):
    avg_loss, acc,f1 = train_epoch_progress(model, train_iter, loss_function, optimizer, text_field, label_field, epoch)
    tqdm.write('Train: loss %.2f acc %.1f f1 %.3f' % (avg_loss, acc*100,f1))
    dev_acc , dev_f1= evaluate(model, dev_iter, loss_function, 'Dev')
    if dev_acc > best_dev_acc:
        if best_dev_acc > 0:
            os.system('rm '+ out_dir + '/best_model' + '.pth')
        best_dev_acc = dev_acc
        best_model = model
        torch.save(best_model.state_dict(), out_dir + '/best_model' + '.pth')
        # evaluate on test with the best dev performance model
        test_acc,test_f1 = evaluate(best_model, test_iter, loss_function, 'Test')
test_acc,test_f1 = evaluate(best_model, test_iter, loss_function, 'Final Test')





Train epoch 1:   0%|          | 0/4049 [00:00<?, ?it/s]

Training...
Writing to /Users/nihaozheng/Desktop/NLP/project/model/pytorch-sentiment-classification-master/runs/1523730993



/Users/nihaozheng/Desktop/NLP/project/model/pytorch-sentiment-classification-master/lstm.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_probs = F.log_softmax(y)
Train epoch 1: 100%|██████████| 4049/4049 [42:42<00:00,  1.58it/s]
/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train: loss 0.89 acc 65.0 f1 0.605


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:89: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


...

Train epoch 2: 100%|██████████| 4049/4049 [1:14:23<00:00,  1.10s/it]


Train: loss 0.48 acc 83.4 f1 0.822
Dev: loss 0.87 acc 67.3 f1 0.66


Train epoch 3:   0%|          | 0/4049 [00:00<?, ?it/s]

Test: loss 0.94 acc 64.7 f1 0.63


Train epoch 3:  53%|█████▎    | 2130/4049 [37:10<33:29,  1.05s/it] 

# try Reg

In [5]:
EPOCHS = 10
USE_GPU = torch.cuda.is_available()
EMBEDDING_DIM = 300
HIDDEN_DIM = 500

BATCH_SIZE = 5
timestamp = str(int(time.time()))
best_dev_acc = 0.0


text_field = data.Field(lower=True)
label_field = data.Field(sequential=False)
train_iter, dev_iter, test_iter = load_sst(text_field, label_field, BATCH_SIZE)






In [6]:
model = BiLSTMSentiment(embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, vocab_size=len(text_field.vocab), label_size=len(label_field.vocab)-1,\
                          use_gpu=USE_GPU, batch_size=BATCH_SIZE)



In [7]:
model.embeddings.weight.data.copy_(torch.from_numpy(pretrained_embeddings))







 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0228 -0.1420 -0.0043  ...   0.1172  0.0888  0.1090
 0.0801  0.1050  0.0498  ...   0.0037  0.0476 -0.0688
          ...             ⋱             ...          
-0.0129 -0.0334  0.0001  ...   0.1511  0.0820 -0.0477
 0.0962  0.1528  0.1690  ...   0.0341  0.2265  0.2421
-0.0400  0.0009 -0.0988  ...   0.0423 -0.0371 -0.0146
[torch.FloatTensor of size 51322x300]

In [8]:
#define optimizer and loss function
# with smaller lr and add regularizaition
best_model = model
optimizer = optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-5)
loss_function = nn.NLLLoss()






In [9]:
print('Training...')
out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
print("Writing to {}\n".format(out_dir))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
for epoch in range(EPOCHS):
    avg_loss, acc,f1 = train_epoch_progress(model, train_iter, loss_function, optimizer, text_field, label_field, epoch)
    tqdm.write('Train: loss %.2f acc %.1f f1 %.3f' % (avg_loss, acc*100,f1))
    dev_acc , dev_f1= evaluate(model, dev_iter, loss_function, 'Dev')
    if dev_acc > best_dev_acc:
        if best_dev_acc > 0:
            os.system('rm '+ out_dir + '/best_model' + '.pth')
        best_dev_acc = dev_acc
        best_model = model
        torch.save(best_model.state_dict(), out_dir + '/best_model' + '.pth')
        # evaluate on test with the best dev performance model
        test_acc,test_f1 = evaluate(best_model, test_iter, loss_function, 'Test')
test_acc,test_f1 = evaluate(best_model, test_iter, loss_function, 'Final Test')







Train epoch 1:   0%|          | 0/4049 [00:00<?, ?it/s]

Training...
Writing to /Users/nihaozheng/Desktop/NLP/project/model/pytorch-sentiment-classification-master/runs/1523837993



/Users/nihaozheng/Desktop/NLP/project/model/pytorch-sentiment-classification-master/bilstm.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_probs = F.log_softmax(y)
Train epoch 1: 100%|██████████| 4049/4049 [2:19:52<00:00,  2.07s/it]


Train: loss 0.87 acc 65.6 f1 0.617


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:89: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

Train: loss 0.48 acc 83.8 f1 0.831
Dev: loss 0.77 acc 69.6 f1 0.68


Train epoch 3:   0%|          | 0/4049 [00:00<?, ?it/s]

Test: loss 0.77 acc 70.0 f1 0.68


Train epoch 3:  71%|███████   | 2869/4049 [2:13:05<54:44,  2.78s/it]  

KeyboardInterrupt: 